In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re

In [ ]:
# Deletes non NA values that have commas in them so that they can be turned into numeric values
def replace_commas(x):
    if pd.isna(x) == False and isinstance(x, (str, int, float)):
        return str(str(x).replace(',', ''))
    return x

In [ ]:
# Deletes % sign from the percentage data so that they can be turned into numeric values
def fix_percentages(x):
    if isinstance(x, (str)):
        return x.replace('%', '')
    return x

In [ ]:
# Formats Zip Codes as Strings by removing ' or adding a 0 in front of a four letter zip code
def fix_zip_code(x):
    if len(str(x)) == 5 and str(x).find("\'") == -1:
        return x
    if isinstance(x, (str)):
        new_zip = x.replace('\'', '')
        if len(new_zip) == 5:
            return new_zip
        return "0" + str(new_zip)
    else:
        return "0" + str(x)

In [85]:
def convert_xlsx_to_csv_and_merge(directory):
    all_data = pd.DataFrame()  # Initialize an empty DataFrame

    for filename in os.listdir(directory):

        # for csvs:
        if filename.endswith('.csv'):
            file_path = os.path.join(directory, filename)
            print(file_path)

            # Read the CSV file and append its contents to the DataFrame
            # Formating Header to remove whitespaces
            df = pd.read_csv(file_path, na_values=['#DIV/0', 'Not Available', ' -   ', ' Not Available ', 'Not applicable to this property type'])
            for col in df.columns:
                new_col = col.strip()
                df.rename(columns = {col:new_col}, inplace = True)

            # Inserting new rows to the dataframe that aren't in some files
            df.insert(17, '% Fuel Oil', 0)
            df.insert(18, '% District Hot Water', 0)
            df.insert(19, '% District Chilled Water', 0)
            df.insert(20, '% Other (Diesel #2, Kerosene, Propane or Other Fuel)', 0)

            # Normalizing the percentages to be decimals adding up to 1 rather than integer%
            if filename.startswith('berdo-2016'):
                df["% Electricity"] = pd.to_numeric(df['% Electricity'].apply(fix_percentages)) / 100
                df["% Gas"] = pd.to_numeric(df["% Gas"].apply(fix_percentages)) / 100
                df["% Steam"] = pd.to_numeric(df["% Steam"].apply(fix_percentages)) / 100

        elif filename.endswith('.xlsx'):
            file_path = os.path.join(directory, filename)
            print(file_path)

            # Read the XLSX file and append its contents to the DataFrame
            # Formating Header to remove whitespaces
            df = pd.read_excel(file_path, na_values=['#DIV/0', 'Not Available', ' -   ', ' Not Available ', 'Not applicable to this property type'])
            for col in df.columns:
                new_col = col.strip()
                df.rename(columns = {col:new_col}, inplace=True)

            # Renaming Onsite Solar in some files to Onsite Renewable (more general and matches rest of files)
            if filename.startswith('berdo-2014'):
                df['Years Reported'] = 2014
                df.rename(columns={'Onsite Solar (kWh)':'Onsite Renewable (kWh)'}, inplace=True)

            if filename.startswith('berdo-2015'):
                df.rename(columns={'Onsite Solar (kWh)':'Onsite Renewable (kWh)'}, inplace=True)

            # Removes unncessary columns on energy action plans (only contains data for 2018, 2019)
            if filename.startswith('berdo-2019') or filename.startswith('berdo-2018'):
                df = df.iloc[:,:-2]

            # Inserting new rows to the dataframe that aren't in some files
            if not (filename.startswith('berdo-2020') or filename.startswith('berdo-2019')):
                df.insert(17, '% Fuel Oil', 0)
                df.insert(18, '% District Hot Water', 0)
                df.insert(19, '% District Chilled Water', 0)
                df.insert(20, '% Other (Diesel #2, Kerosene, Propane or Other Fuel)', 0)
            elif filename.startswith('berdo-2019'):
                df.insert(20, '% Other (Diesel #2, Kerosene, Propane or Other Fuel)', 0)
            else:
                df['Years Reported'] = 2020

            # Formatting percentages to dtype float64
            df["% Electricity"] = pd.to_numeric(df['% Electricity'].apply(fix_percentages))
            df["% Gas"] = pd.to_numeric(df["% Gas"].apply(fix_percentages))
            df["% Steam"] = pd.to_numeric(df["% Steam"].apply(fix_percentages))

            # Normalizing the percentages to be decimals adding up to 1 rather than integer%
            if filename.startswith('berdo-2014'):
                df["% Electricity"] = df['% Electricity'].apply(fix_percentages) / 100
                df["% Gas"] = df["% Gas"].apply(fix_percentages) / 100
                df["% Steam"] = df["% Steam"].apply(fix_percentages) / 100

        # Add Year column before appending and getting year value using regex
        regex = r'\b(\d{4})\b'
        match = re.search(regex, filename)
        if match:
            year = match.group(1)
            print(year)

        df['Year'] = year

        # All the formatting and cleaning using above functions to
        # change select features into correct dtypes
        df.ZIP = df.ZIP.apply(fix_zip_code)
        new_series = df['Onsite Renewable (kWh)']
        new_series = new_series.apply(replace_commas)
        df['Onsite Renewable (kWh)'] = pd.to_numeric(new_series)
        df['Onsite Renewable (kWh)'] = df['Onsite Renewable (kWh)'].replace(np.nan, 0)
        df['Gross Area (sq ft)'] = df['Gross Area (sq ft)'].apply(replace_commas)
        df['Gross Area (sq ft)'] = pd.to_numeric(df['Gross Area (sq ft)'])
        df.Year = pd.to_numeric(df.Year)
        df['Energy Star Certified'] = df['Energy Star Certified'].replace(np.nan, 0)
        df['Energy Star Score'] = pd.to_numeric(df['Energy Star Score'])
        df['Energy Star Score'] = df['Energy Star Score'].replace(np.nan, 0)
        df['GHG Emissions (MTCO2e)'] = df['GHG Emissions (MTCO2e)'].apply(replace_commas)
        df['GHG Emissions (MTCO2e)'] = pd.to_numeric(df['GHG Emissions (MTCO2e)'])
        df['GHG Intensity (kgCO2/sf)'] = df['GHG Intensity (kgCO2/sf)'].apply(replace_commas)
        df['GHG Intensity (kgCO2/sf)'] = pd.to_numeric(df['GHG Intensity (kgCO2/sf)'])
        df['Total Site Energy (kBTU)'] = df['Total Site Energy (kBTU)'].apply(replace_commas)
        df['Total Site Energy (kBTU)'] = pd.to_numeric(df['Total Site Energy (kBTU)'])

        all_data = pd.concat([all_data, df], ignore_index=True, axis=0)

    return all_data

# Specify the directory containing the XLSX and CSV files
directory_path = 'berdo'

# Call the function to convert XLSX files to CSV and merge them into a single DataFrame
merged_data = convert_xlsx_to_csv_and_merge(directory_path)

berdo\berdo-2014.xlsx
2014
berdo\berdo-2015.xlsx
2015
berdo\berdo-2016.csv
2016
berdo\berdo-2017.csv
2017
berdo\berdo-2018.xlsx
2018
berdo\berdo-2019.xlsx
2019
berdo\berdo-2020.xlsx
2020


In [92]:
merged_data.head(5)

,Property Name,Reported,Property Type,Address,ZIP,Gross Area (sq ft),Site EUI (kBTU/sf),Energy Star Score,Energy Star Certified,Property Uses,...,% District Hot Water,% District Chilled Water,"% Other (Diesel #2, Kerosene, Propane or Other Fuel)",Water Intensity (gal/sf),Onsite Renewable (kWh),User Submitted Info,User Submitted Link,Tax Parcel,Years Reported,Year
0,MEEI -Longwood,Yes,Ambulatory Surgical Center,800 Huntington Ave,02115,76300.0,173.1,0.0,0,Ambulatory Surgical Center,...,0.0,0.0,0.0,NaN,0.0,NaN,NaN,1000894000,2014,2014
1,Prime Motor Group,Yes,Automobile Dealership,1525-1607 VFW Parkway,02132,150000.0,28.7,0.0,0,"Automobile Dealership, Parking",...,0.0,0.0,0.0,8.957333,0.0,NaN,NaN,2010643010,2014,2014
2,New England Center for Homeless Veterans,Yes,Barracks,17 Court St.,02108,130000.0,49.8,58.0,0,Barracks,...,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0302886000,2014,2014
3,Dental Health Building,Yes,College/University,1 Kneeland Street,02111,330486.0,284.7,0.0,0,"College/University, Hospital (General Medical ...",...,0.0,0.0,0.0,24.437646,0.0,"\n\nWater, electricity and steam come from Tuf...",NaN,0305380000,2014,2014
4,Posner Hall,Yes,College/University,200 Harrison Avenue,02111,60284.0,95.9,0.0,0,College/University,...,0.0,0.0,0.0,20.015261,0.0,NaN,NaN,0305081000,2014,2014


In [88]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18225 entries, 0 to 18224
Data columns (total 28 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Property Name                                         15751 non-null  object 
 1   Reported                                              18225 non-null  object 
 2   Property Type                                         18153 non-null  object 
 3   Address                                               18167 non-null  object 
 4   ZIP                                                   18225 non-null  object 
 5   Gross Area (sq ft)                                    18068 non-null  float64
 6   Site EUI (kBTU/sf)                                    13456 non-null  float64
 7   Energy Star Score                                     18225 non-null  float64
 8   Energy Star Certified                                 18

In [89]:
merged_data.to_csv("berdo-better.csv", sep=',', encoding='utf-8')